In [1]:
import pandas as pd
import numpy as np
import re
import time
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore")

In [2]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
tic = time.time()
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']
    
invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']
    
question_info = pd.read_table(os.path.join(DATA_PATH, 'question_info_0926.txt', header=None)
question_info.columns = ['问题ID',  '问题创建时间' , '问题标题的单字编码序列' , '问题标题的切词编码序列' , '问题描述的单字编码序列',  '问题描述的词编码序列' , '问题绑定的话题ID']

oversample = False

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)
    
data = data.merge(question_info, on='问题ID', how='left')
print("Used time: %d s" % (time.time()-tic))

Used time: 52 s


In [3]:
tic = time.time()
tmp = data['邀请创建时间'].apply(lambda x : x.split('-'))
data['邀请创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['邀请创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))
    
tmp = data['问题创建时间'].apply(lambda x : x.split('-'))
data['问题创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['问题创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))
    
data['邀请创建时间_DH'] = data['邀请创建时间_D']*24 + data['邀请创建时间_H']
data['问题创建时间_DH'] = data['问题创建时间_D']*24 + data['问题创建时间_H']
    
tmp = data.groupby(['用户ID'])['邀请创建时间_DH'].agg({'邀请创建时间_mean':'mean','邀请创建时间_max':'max','邀请创建时间_min':'min','邀请创建时间_std':'std'})
data = data.merge(tmp, on='用户ID', how='left')
    
tmp = data.groupby(['用户ID'])['问题创建时间_DH'].agg({'问题创建时间_mean':'mean','问题创建时间_max':'max','问题创建时间_min':'min','问题创建时间_std':'std'})
data = data.merge(tmp, on='用户ID', how='left')
    
tmp = data[['邀请创建时间_mean','邀请创建时间_max','邀请创建时间_min','邀请创建时间_std','问题创建时间_mean','问题创建时间_max','问题创建时间_min','问题创建时间_std']]
print("Used time: %d s" % (time.time()-tic))

Used time: 168 s


In [4]:
tmp.to_pickle(os.path.join(FEAT_PATH, './features/invite_time_stats.pickle'))
print("Feature Saved, shape:",tmp.shape)

Feature Saved, shape: (10630880, 8)
